In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Kaggle/'

In [ ]:
import pandas as pd
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
import pickle
import gc

In [ ]:
train = pd.read_csv(path+'train.csv')
test_unlabeled = pd.read_csv(path+'test_unlabeled.csv')


In [ ]:
filenames = !ls {path}chunk*.csv
filenames

['/content/drive/MyDrive/Kaggle/chunk_10.csv',
 '/content/drive/MyDrive/Kaggle/chunk_11.csv',
 '/content/drive/MyDrive/Kaggle/chunk_12.csv',
 '/content/drive/MyDrive/Kaggle/chunk_13.csv',
 '/content/drive/MyDrive/Kaggle/chunk_14.csv',
 '/content/drive/MyDrive/Kaggle/chunk_15.csv',
 '/content/drive/MyDrive/Kaggle/chunk_16.csv',
 '/content/drive/MyDrive/Kaggle/chunk_17.csv',
 '/content/drive/MyDrive/Kaggle/chunk_18.csv',
 '/content/drive/MyDrive/Kaggle/chunk_19.csv',
 '/content/drive/MyDrive/Kaggle/chunk_1.csv',
 '/content/drive/MyDrive/Kaggle/chunk_20.csv',
 '/content/drive/MyDrive/Kaggle/chunk_21.csv',
 '/content/drive/MyDrive/Kaggle/chunk_22.csv',
 '/content/drive/MyDrive/Kaggle/chunk_23.csv',
 '/content/drive/MyDrive/Kaggle/chunk_24.csv',
 '/content/drive/MyDrive/Kaggle/chunk_25.csv',
 '/content/drive/MyDrive/Kaggle/chunk_26.csv',
 '/content/drive/MyDrive/Kaggle/chunk_27.csv',
 '/content/drive/MyDrive/Kaggle/chunk_28.csv',
 '/content/drive/MyDrive/Kaggle/chunk_29.csv',
 '/content/dri

In [ ]:
train.head()

,Unnamed: 0,mint,slot_min,slot_graduated,has_graduated,is_valid
0,1,BmTDA5HqcemLkEgpyK25sDhbvk652CTXjdWEa8fLpump,317876496,NaN,False,True
1,3,4FJwryCAMMePNeWw9LTBXXfXABdkKcAxkTT6h9pdpump,317876500,NaN,False,True
2,4,DeXchZLMzFm9nfvkfTKiXvBWWEpGMF6VDzvbMB6mpump,317876500,NaN,False,True
3,5,6Mx4fnEPWpRLKtxY8VzVXo3CWwu2dvGp9BSJMWfVpump,317876500,NaN,False,True
4,6,4rbGH5peYV3FvCfncfsfwtBNwpjiG36nZtvxoPZ9pump,317876500,NaN,False,True


In [ ]:
df_sample_chunk_head = pd.read_csv(filenames[1], nrows=5)
# print(df_sample_chunk_head)
print(df_sample_chunk_head.columns.tolist())

['block_time', 'slot', 'tx_idx', 'signing_wallet', 'direction', 'base_coin', 'base_coin_amount', 'quote_coin_amount', 'virtual_token_balance_after', 'virtual_sol_balance_after', 'signature', 'provided_gas_fee', 'provided_gas_limit', 'fee', 'consumed_gas']


In [ ]:
df_sample_chunk_head = pd.read_csv(path+'token_info_onchain_divers_v2.csv', nrows=5)
# print(df_sample_chunk_head)
print(df_sample_chunk_head.columns.tolist())

['block_time', 'slot', 'tx_idx', 'creator', 'name', 'symbol', 'url', 'mint', 'bundle_size', 'gas_used', 'amount_of_instructions', 'amount_of_lookup_reads', 'amount_of_lookup_writes', 'bundle_structure', 'bundled_buys', 'bundled_buys_count', 'dev_balance', 'creation_ix_index', 'curve_address', 'pf_program_index', 'direct_pf_invocation', 'version']


In [ ]:
MAX_SLOT_OFFSET = 100
metadata_mint_col = 'mint'
metadata_creation_slot_col = 'slot'
train_test_mint_col = 'mint'

metadata_filename = 'token_info_onchain_divers_v2.csv'
train_filename = 'train.csv'
test_filename = 'test_unlabeled.csv'

In [ ]:
train_df = pd.read_csv(path+train_filename, usecols=[train_test_mint_col])
test_df = pd.read_csv(path+test_filename, usecols=[train_test_mint_col])
metadata_df = pd.read_csv(path+metadata_filename, usecols=[metadata_mint_col, metadata_creation_slot_col])

Combine and Dedupe the Mints

In [ ]:
all_mints = pd.concat([train_df[[train_test_mint_col]], test_df[[train_test_mint_col]]], ignore_index=True)
unique_mints_df = all_mints.drop_duplicates().reset_index(drop=True)
print(f"Total unique mints from train and test sets: {len(unique_mints_df)}")

Total unique mints from train and test sets: 1118389


Merge creation slots

In [ ]:
metadata_df = metadata_df.drop_duplicates(subset=metadata_mint_col)

# Merge metadata (creation slot) onto the unique mint list
# Use left merge to keep all unique mints, even if metadata is missing for some
mints_info_target = pd.merge(
    unique_mints_df,
    metadata_df,
    left_on=train_test_mint_col, # 'mint' from unique_mints_df
    right_on=metadata_mint_col, # 'mint' from metadata_df
    how='left'
)

In [ ]:
print(f"Shape after merging metadata: {mints_info_target.shape}")

missing_slots = mints_info_target[metadata_creation_slot_col].isnull().sum()
print(f"\nNumber of mints missing creation slot ('{metadata_creation_slot_col}') info: {missing_slots}")

Shape after merging metadata: (1118389, 2)

Number of mints missing creation slot ('slot') info: 55165


Dropping missing creation splots

In [ ]:
mints_info_target = mints_info_target.dropna(subset=[metadata_creation_slot_col]).copy()
print(f"Proceeding with {len(mints_info_target)} mints that have creation slot information.")

Proceeding with 1063224 mints that have creation slot information.


Calculate the slot ranges

In [ ]:
# Ensure creation slot is integer type (originally was int64)
mints_info_target[metadata_creation_slot_col] = mints_info_target[metadata_creation_slot_col].astype(int)

In [ ]:
# Define the start and end slot for the feature window
mints_info_target['target_slot_start'] = mints_info_target[metadata_creation_slot_col]
mints_info_target['target_slot_end'] = mints_info_target[metadata_creation_slot_col] + MAX_SLOT_OFFSET

In [ ]:
# Clean up columns - keep only necessary info for processing chunks
mints_info_target = mints_info_target[[
    train_test_mint_col, # The actual mint address
    'target_slot_start',
    'target_slot_end'
]].rename(columns={train_test_mint_col: 'mint'}) # Standardize mint column name to 'mint'

In [ ]:
print(f"Shape of final DataFrame: {mints_info_target.shape}")
print(mints_info_target.head())
print(mints_info_target.info())

Shape of final DataFrame: (1063224, 3)
                                           mint  target_slot_start  \
0  BmTDA5HqcemLkEgpyK25sDhbvk652CTXjdWEa8fLpump          317876496   
1  4FJwryCAMMePNeWw9LTBXXfXABdkKcAxkTT6h9pdpump          317876500   
2  DeXchZLMzFm9nfvkfTKiXvBWWEpGMF6VDzvbMB6mpump          317876500   
3  6Mx4fnEPWpRLKtxY8VzVXo3CWwu2dvGp9BSJMWfVpump          317876500   
4  4rbGH5peYV3FvCfncfsfwtBNwpjiG36nZtvxoPZ9pump          317876500   

   target_slot_end  
0        317876596  
1        317876600  
2        317876600  
3        317876600  
4        317876600  
<class 'pandas.core.frame.DataFrame'>
Index: 1063224 entries, 0 to 1118388
Data columns (total 3 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   mint               1063224 non-null  object
 1   target_slot_start  1063224 non-null  int64 
 2   target_slot_end    1063224 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 32.4+ MB
None


In [ ]:
mints_info_target_save_path = os.path.join(path, 'mints_info_target_prepared.parquet')

try:
    mints_info_target.to_parquet(mints_info_target_save_path) # Keep index by default
    print(f"Successfully saved mints_info_target to: {mints_info_target_save_path}")
except Exception as e:
    print(f"ERROR saving mints_info_target: {e}")
# --- End Save ---

Successfully saved mints_info_target to: /content/drive/MyDrive/Kaggle/mints_info_target_prepared.parquet


# Read for aggregation and feature engineering

We are transforming the raw, time-series transaction data from the chunks into a fixed set of summary statistics (features) for each mint, based only on its first 100 slots of activity.

In [ ]:
chunk_files = sorted(glob.glob(os.path.join(path, 'chunk_*.csv')))
chunk_files

['/content/drive/MyDrive/Kaggle/chunk_1.csv',
 '/content/drive/MyDrive/Kaggle/chunk_10.csv',
 '/content/drive/MyDrive/Kaggle/chunk_11.csv',
 '/content/drive/MyDrive/Kaggle/chunk_12.csv',
 '/content/drive/MyDrive/Kaggle/chunk_13.csv',
 '/content/drive/MyDrive/Kaggle/chunk_14.csv',
 '/content/drive/MyDrive/Kaggle/chunk_15.csv',
 '/content/drive/MyDrive/Kaggle/chunk_16.csv',
 '/content/drive/MyDrive/Kaggle/chunk_17.csv',
 '/content/drive/MyDrive/Kaggle/chunk_18.csv',
 '/content/drive/MyDrive/Kaggle/chunk_19.csv',
 '/content/drive/MyDrive/Kaggle/chunk_2.csv',
 '/content/drive/MyDrive/Kaggle/chunk_20.csv',
 '/content/drive/MyDrive/Kaggle/chunk_21.csv',
 '/content/drive/MyDrive/Kaggle/chunk_22.csv',
 '/content/drive/MyDrive/Kaggle/chunk_23.csv',
 '/content/drive/MyDrive/Kaggle/chunk_24.csv',
 '/content/drive/MyDrive/Kaggle/chunk_25.csv',
 '/content/drive/MyDrive/Kaggle/chunk_26.csv',
 '/content/drive/MyDrive/Kaggle/chunk_27.csv',
 '/content/drive/MyDrive/Kaggle/chunk_28.csv',
 '/content/driv

In [ ]:
df_sample_chunk_head = pd.read_csv(filenames[1], nrows=5)
print(df_sample_chunk_head)

            block_time       slot  tx_idx  \
0  2025-02-07 19:37:38  319209603      29   
1  2025-02-07 19:37:38  319209603     389   
2  2025-02-07 19:37:38  319209603      92   
3  2025-02-07 19:37:38  319209603     154   
4  2025-02-07 19:37:38  319209603    1260   

                                 signing_wallet direction  \
0  Ea8wdeVtJKvDxBFditrWsK2mfibnLRAea5MN2zDHAyp5       buy   
1  5YET3YapxD6to6rqPqTWB3R9pSbURy6yduuUtoZkzoPX       buy   
2  E4Q4x2R59yrA39uSdfBW7GycCiP67yhVWsybdZtqMLQa       buy   
3  ATtBj32ooZb3PToqTjV8rBW3XtUEp8xXWmv6LZR3PrRZ       buy   
4   o7RY6P2vQMuGSu1TrLM81weuzgDjaCRTXYRaXJwWcvc       buy   

                                      base_coin  base_coin_amount  \
0  8u2XH268edziKqBRNXKReVBnqMupiRL3w7RWAU8rpump      164980128238   
1  8u2XH268edziKqBRNXKReVBnqMupiRL3w7RWAU8rpump      341357517312   
2  BF6wPpSPpWEbEq3wcyfnmwDjYXGa443DhvkyNiitpump      352511236077   
3  H6mFjATx2yx8NQtYuzf6KfYuu3crDLTtn2M5Y5G8pump    34612903225806   
4  H6mFjATx2yx8NQ

In [ ]:
COLS_TO_LOAD_FROM_CHUNKS = [
    'block_time', # Timestamp (useful for time-based features)
    'slot', # The block/slot number
    'tx_idx',
    'signing_wallet', # Wallet initiating the swap
    'direction',  # 'buy' or 'sell'
    'base_coin', # The mint address
    'base_coin_amount',
    'quote_coin_amount', # SOL amount involved
    'virtual_token_balance_after', # State of the virtual LP
    'virtual_sol_balance_after',
    'signature',
    'provided_gas_fee',
    'provided_gas_limit',
    'fee',
    'consumed_gas'
  ]

**Phase 1: Analyzing Column Potential (Step-by-Step Thinking)**

We'll examine each column from `COLS_TO_LOAD_FROM_CHUNKS` and brainstorm its potential value within the "first 100 slots" constraint:

1.  **`block_time` (Timestamp):**
    *   **Direct Use:** Can calculate the precise duration (in seconds) of activity within the 100-slot window (`last_time - first_time`).
    *   **Engineered Features:**
        *   `time_span_seconds_100s`: The real-world time duration of observed activity. Short spans might indicate bursts.
        *   `tx_per_second_100s`: Transaction velocity (`tx_count / time_span_seconds`). High values suggest intense early interest.
        *   `sol_volume_per_second_100s`: Economic velocity.
        *   Time-weighted features (advanced): Give more weight to earlier or later transactions within the window.
    *   **Potential Signal:** Measures real-world speed of interaction, complementing the block-based `slot` timing. Captures potential bursts of activity that might occur within a single slot.

2.  **`slot` (Block/Slot Number):**
    *   **Direct Use:** Essential for filtering transactions to the `[creation_slot, creation_slot + 100)` window.
    *   **Engineered Features:**
        *   `first_slot_offset_100s`: `first_observed_slot - creation_slot`. How quickly did activity *start* after creation? (Value near 0 is expected if active immediately).
        *   `last_slot_offset_100s`: `last_observed_slot - creation_slot`. How long did activity persist within the window? (Value near 99 suggests sustained activity).
        *   `slot_span_100s`: `last_observed_slot - first_observed_slot`. How many slots saw activity?
        *   `tx_per_slot_100s`: Density of transactions across active slots (`tx_count / (slot_span + 1)`).
        *   `activity_slot_density_100s`: `(slot_span + 1) / 100`. What fraction of the window saw *any* activity?
    *   **Potential Signal:** Measures activity timing and duration relative to blockchain blocks.

3.  **`tx_idx` (Transaction Index within Slot):**
    *   **Direct Use:** Indicates order within a slot.
    *   **Engineered Features:**
        *   `avg_tx_idx_100s`: Average position within a block. Consistently high values might indicate getting included later in blocks.
        *   `max_tx_idx_100s`: Maximum index seen. Could suggest high contention slots.
        *   `stddev_tx_idx_100s`: Variability in transaction position.
    *   **Potential Signal:** Subtle signal, might reflect transaction priority, network congestion during interaction, or bot behavior landing transactions at specific points in blocks. Likely less impactful than volume/wallet features.

4.  **`signing_wallet` (Wallet Initiating Swap):**
    *   **Direct Use:** Identifier for the actor.
    *   **Engineered Features:**
        *   `unique_wallets_count_100s`: Total distinct participants. Key indicator of community size/interest.
        *   `unique_buyers_count_100s`: Distinct wallets performing 'buy'.
        *   `unique_sellers_count_100s`: Distinct wallets performing 'sell'.
        *   `new_wallet_rate_100s`: `unique_wallets_count / slot_span` or `/ time_span_seconds`. How quickly are new participants joining?
        *   `avg_tx_per_wallet_100s`: `tx_count / unique_wallets_count`. High value might indicate bots or very active individuals.
        *   `avg_sol_vol_per_wallet_100s`: `total_sol_volume / unique_wallets_count`.
        *   `concentration_metrics (Advanced)`: E.g., % of volume from top 1/3/5 wallets. Requires storing per-wallet data, memory-intensive. Start without, add later if needed.
        *   `creator_activity (Advanced)`: Requires linking `signing_wallet` to the `creator` from metadata. Did the creator buy/sell early? Complex join.
    *   **Potential Signal:** Extremely important. Reflects breadth and depth of engagement, potential bot activity, and overall community interest.

5.  **`direction` ('buy' or 'sell'):**
    *   **Direct Use:** Categorical flag for transaction type.
    *   **Engineered Features:** (Many features depend on this)
        *   `buy_tx_count_100s`, `sell_tx_count_100s`.
        *   `buy_sell_tx_ratio_100s`: `buy_count / sell_count`. High ratio indicates buying pressure.
        *   Dominant direction features: Was the first/last tx a buy or sell? Percentage of buys.
    *   **Potential Signal:** Fundamental indicator of market sentiment within the crucial early window.

6.  **`base_coin` (Token Mint Address):**
    *   **Direct Use:** The primary key for grouping transactions per token. Not a feature itself.

7.  **`base_coin_amount` (Token Amount):**
    *   **Direct Use:** Quantity of the memecoin traded.
    *   **Engineered Features:**
        *   `buy_token_vol_100s`, `sell_token_vol_100s`, `total_token_vol_100s`, `net_token_vol_100s`.
        *   `avg_buy_token_100s`, `avg_sell_token_100s`, `avg_tx_token_100s`.
        *   `stddev_token_vol_100s` (Advanced, if memory allows).
        *   Used in implied price calculations.
    *   **Potential Signal:** Reflects activity in terms of the token itself. Large average sizes might indicate whales. Complements SOL volume.

8.  **`quote_coin_amount` (SOL Amount):**
    *   **Direct Use:** SOL value exchanged. Primary economic indicator.
    *   **Engineered Features:**
        *   `buy_sol_vol_100s`, `sell_sol_vol_100s`, `total_sol_vol_100s`, `net_sol_vol_100s` (BUY - SELL, critical measure of net inflow).
        *   `avg_buy_sol_100s`, `avg_sell_sol_100s`, `avg_tx_sol_100s`.
        *   `stddev_sol_vol_100s` (Advanced, if memory allows).
        *   `max_buy_sol_tx_100s`, `max_sell_sol_tx_100s`: Size of largest individual buy/sell.
    *   **Potential Signal:** Highly important. Directly measures the SOL flowing into/out of the token, reflecting demand and perceived value. Net volume is key.

9.  **`virtual_token_balance_after` (Token side of LP):**
    *   **Direct Use:** State of the token pool after the transaction.
    *   **Engineered Features:**
        *   `lp_token_start_100s`, `lp_token_end_100s`.
        *   `lp_token_change_100s`: `end - start`.
        *   `lp_token_change_pct_100s`: `(end - start) / start`.
        *   `lp_token_min_100s`, `lp_token_max_100s`: Range of LP token balance during the window.
        *   Used in implied price calculations.
    *   **Potential Signal:** Tracks the supply dynamics within the bonding curve's virtual pool.

10. **`virtual_sol_balance_after` (SOL side of LP):**
    *   **Direct Use:** State of the SOL pool after the transaction. Crucial liquidity indicator.
    *   **Engineered Features:**
        *   `lp_sol_start_100s`, `lp_sol_end_100s`. (End value is a strong candidate feature itself).
        *   `lp_sol_change_100s`: `end - start`. Key indicator of net liquidity change.
        *   `lp_sol_change_pct_100s`: `(end - start) / start`. Relative growth.
        *   `lp_sol_min_100s`, `lp_sol_max_100s`: Range of SOL liquidity during the window.
        *   `rate_lp_sol_change_100s`: `lp_sol_change / slot_span` or `/ time_span_seconds`. How fast is liquidity accumulating?
    *   **Potential Signal:** Extremely important. Directly tracks the accumulation of SOL liquidity, which is the basis for graduation. The ending value and the change are critical.

11. **`signature` (Transaction Signature):**
    *   **Direct Use:** Unique identifier.
    *   **Engineered Features:** Could potentially count unique signatures if `tx_count` is suspected to double-count retry attempts, but unlikely necessary.
    *   **Potential Signal:** Low direct value for prediction.

12. **`provided_gas_fee`, `provided_gas_limit`, `fee`, `consumed_gas`:**
    *   **Direct Use:** Details about transaction costs.
    *   **Engineered Features:**
        *   `total_gas_consumed_100s`, `avg_gas_consumed_100s`, `stddev_gas_consumed_100s`.
        *   `total_fee_100s`, `avg_fee_100s`, `stddev_fee_100s`.
        *   `avg_gas_efficiency_100s`: `avg(consumed / provided_limit)`.
        *   `max_fee_100s`, `min_fee_100s`.
    *   **Potential Signal:** Might reflect user sophistication, urgency (high fees for priority), or bot patterns (consistent fees/gas usage). Potentially subtle signals.


In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from tqdm.notebook import tqdm
import time
import warnings

In [ ]:
# Suppress division by zero warnings during calculation, handle with np.inf/nan
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [ ]:
path = '/content/drive/MyDrive/Kaggle/'
chunk_files = sorted(glob.glob(os.path.join(path, 'chunk_*.csv')))

In [ ]:
COLS_TO_LOAD_FROM_CHUNKS = [
    'block_time', 'slot', 'tx_idx', 'signing_wallet', 'direction', 'base_coin',
    'base_coin_amount', 'quote_coin_amount', 'virtual_token_balance_after',
    'virtual_sol_balance_after', 'signature', 'provided_gas_fee',
    'provided_gas_limit', 'fee', 'consumed_gas'
]

In [ ]:
# --- Step 1: Prepare Mint Lookup ---
target_mints_dict = mints_info_target.set_index('mint').to_dict('index')
print(f"Lookup dictionary created for {len(target_mints_dict)} mints.")

Lookup dictionary created for 1063224 mints.


In [ ]:
# --- Step 2: Initialize Feature Storage ---
# This dictionary structure is defined *inside* the loop when a mint is first seen.
# The keys listed here are just for reference of what will be created.
_initial_feature_structure_template = {
    'mint': '', # Set to the mint address
    'tx_count_100s': 0,
    'buy_tx_count_100s': 0, 'sell_tx_count_100s': 0,
    'total_sol_vol_100s': 0.0, 'buy_sol_vol_100s': 0.0, 'sell_sol_vol_100s': 0.0,
    'total_token_vol_100s': 0.0, 'buy_token_vol_100s': 0.0, 'sell_token_vol_100s': 0.0,
    'total_gas_consumed_100s': 0.0, 'total_fee_100s': 0.0,
    'unique_wallets_set': set(), 'unique_buyers_set': set(), 'unique_sellers_set': set(),
    'first_slot_100s': -1, 'last_slot_100s': -1, # Placeholder, set on first tx
    'first_time_100s': pd.NaT, 'last_time_100s': pd.NaT, # Placeholder, set on first tx
    'lp_sol_start_100s': 0.0, 'lp_sol_end_100s': 0.0,
    'lp_token_start_100s': 0.0, 'lp_token_end_100s': 0.0,
    'min_lp_sol_100s': np.inf, 'max_lp_sol_100s': -np.inf, # Initialize correctly for min/max
    'min_lp_token_100s': np.inf, 'max_lp_token_100s': -np.inf,
    'sum_tx_idx_100s': 0, 'max_tx_idx_100s': -1,
    'max_buy_sol_tx_100s': 0.0, 'max_sell_sol_tx_100s': 0.0,

    # --- NEW V2 Features Initialization ---
    # Half-window accumulators
    'tx_count_first_half': 0,
    'sol_vol_first_half': 0.0,
    'wallets_first_half_set': set(),
    'lp_sol_at_half_way_mark': 0.0, # LP SOL balance after last tx in first half
    'tx_count_second_half': 0,
    'sol_vol_second_half': 0.0,
    'wallets_second_half_set': set(), # Will contain wallets from both halves, need to diff later

    # Gas limit accumulator
    'total_provided_gas_limit_100s': 0.0,

    # Placeholder for volatility lists (if implemented later)
    # 'tx_sol_amounts_list': [],
    # 'lp_sol_values_list': [],
}

features_per_mint = {} # This will store the actual data, keyed by mint

In [ ]:
import pandas as pd
import numpy as np
import time
import os
from tqdm.auto import tqdm
import gc
# import pickle # No longer used for main data transfer between step 3 and 4
import pyarrow as pa
import pyarrow.parquet as pq

# --- Helper function for safe division (used in feature finalization) ---
# This should be identical to your original safe_divide function
def safe_divide(numerator, denominator, default=np.nan):
    if denominator is None or pd.isna(denominator) or denominator == 0:
        return default
    try:
        result = numerator / denominator
        # Replace potential inf/-inf resulting from very small denominators too
        return result if np.isfinite(result) else default
    except (ZeroDivisionError, TypeError, ValueError):
        return default

# --- Helper function for finalizing features for a single mint ---
# This function encapsulates the logic from your original Step 4's main loop
def finalize_mint_features_dict(mint_key, features_data, creation_slots_map_param):
    state = features_data.copy() # Work on a copy
    tx_count = state.get('tx_count_100s', 0)

    if tx_count == 0:
        return None # Skip mints with no transactions in the window

    # --- Basic Features (Counts, Averages, Net, Ratios) ---
    state['unique_wallets_count_100s'] = len(state.get('unique_wallets_set', set()))
    state['unique_buyers_count_100s'] = len(state.get('unique_buyers_set', set()))
    state['unique_sellers_count_100s'] = len(state.get('unique_sellers_set', set()))

    state['avg_tx_idx_100s'] = safe_divide(state.get('sum_tx_idx_100s', 0), tx_count, default=0.0)
    state['avg_gas_consumed_100s'] = safe_divide(state.get('total_gas_consumed_100s', 0.0), tx_count, default=0.0)
    state['avg_fee_100s'] = safe_divide(state.get('total_fee_100s', 0.0), tx_count, default=0.0)
    state['avg_sol_vol_per_tx_100s'] = safe_divide(state.get('total_sol_vol_100s', 0.0), tx_count, default=0.0)
    state['avg_token_vol_per_tx_100s'] = safe_divide(state.get('total_token_vol_100s', 0.0), tx_count, default=0.0)
    state['avg_buy_sol_100s'] = safe_divide(state.get('buy_sol_vol_100s', 0.0), state.get('buy_tx_count_100s', 0), default=0.0)
    state['avg_sell_sol_100s'] = safe_divide(state.get('sell_sol_vol_100s', 0.0), state.get('sell_tx_count_100s', 0), default=0.0)
    state['avg_buy_token_100s'] = safe_divide(state.get('buy_token_vol_100s', 0.0), state.get('buy_tx_count_100s', 0), default=0.0)
    state['avg_sell_token_100s'] = safe_divide(state.get('sell_token_vol_100s', 0.0), state.get('sell_tx_count_100s', 0), default=0.0)
    state['avg_sol_vol_per_wallet_100s'] = safe_divide(state.get('total_sol_vol_100s', 0.0), state['unique_wallets_count_100s'], default=0.0)
    state['avg_tx_per_wallet_100s'] = safe_divide(tx_count, state['unique_wallets_count_100s'], default=0.0)

    state['net_sol_vol_100s'] = state.get('buy_sol_vol_100s', 0.0) - state.get('sell_sol_vol_100s', 0.0)
    state['net_token_vol_100s'] = state.get('buy_token_vol_100s', 0.0) - state.get('sell_token_vol_100s', 0.0)

    state['buy_sell_tx_ratio_100s'] = safe_divide(state.get('buy_tx_count_100s', 0), state.get('sell_tx_count_100s', 0))
    state['buy_sell_sol_vol_ratio_100s'] = safe_divide(state.get('buy_sol_vol_100s', 0.0), state.get('sell_sol_vol_100s', 0.0))
    state['unique_buyers_ratio_100s'] = safe_divide(state['unique_buyers_count_100s'], state['unique_wallets_count_100s'], default=0.0)
    state['unique_sellers_ratio_100s'] = safe_divide(state['unique_sellers_count_100s'], state['unique_wallets_count_100s'], default=0.0)

    # --- LP State Features ---
    lp_sol_start = state.get('lp_sol_start_100s', 0.0)
    lp_sol_end = state.get('lp_sol_end_100s', 0.0)
    lp_token_start = state.get('lp_token_start_100s', 0.0)
    lp_token_end = state.get('lp_token_end_100s', 0.0)
    state['lp_sol_change_100s'] = lp_sol_end - lp_sol_start
    state['lp_token_change_100s'] = lp_token_end - lp_token_start
    state['lp_sol_change_pct_100s'] = safe_divide(state['lp_sol_change_100s'], lp_sol_start)
    state['lp_token_change_pct_100s'] = safe_divide(state['lp_token_change_100s'], lp_token_start)

    # --- Implied Price Features ---
    state['price_start_100s'] = safe_divide(lp_sol_start, lp_token_start)
    state['price_end_100s'] = safe_divide(lp_sol_end, lp_token_end)
    if pd.notna(state['price_start_100s']) and pd.notna(state['price_end_100s']):
        state['price_change_100s'] = state['price_end_100s'] - state['price_start_100s']
        state['price_change_pct_100s'] = safe_divide(state['price_change_100s'], state['price_start_100s'])
    else:
        state['price_change_100s'] = np.nan
        state['price_change_pct_100s'] = np.nan
    state['avg_price_per_buy_tx_100s'] = safe_divide(state.get('buy_sol_vol_100s', 0.0), state.get('buy_token_vol_100s', 0.0))
    state['avg_price_per_sell_tx_100s'] = safe_divide(state.get('sell_sol_vol_100s', 0.0), state.get('sell_token_vol_100s', 0.0))

    # --- Timing and Span Features ---
    first_slot_100s = state.get('first_slot_100s', 0) # Ensure a default if somehow not set
    last_slot_100s = state.get('last_slot_100s', first_slot_100s)
    state['slot_span_100s'] = last_slot_100s - first_slot_100s

    creation_slot = creation_slots_map_param.get(mint_key, first_slot_100s) # Use param name
    state['first_slot_offset_100s'] = first_slot_100s - creation_slot
    state['last_slot_offset_100s'] = last_slot_100s - creation_slot

    if pd.notna(state.get('first_time_100s')) and pd.notna(state.get('last_time_100s')) and state['last_time_100s'] > state['first_time_100s']:
        time_diff = state['last_time_100s'] - state['first_time_100s']
        state['time_span_seconds_100s'] = time_diff.total_seconds()
    else:
         state['time_span_seconds_100s'] = 0.0

    # --- Rate Features ---
    slot_span_eff = state['slot_span_100s'] + 1
    time_span_eff = state['time_span_seconds_100s'] if state['time_span_seconds_100s'] > 0 else 1.0

    state['tx_per_slot_100s'] = safe_divide(tx_count, slot_span_eff, default=0.0)
    state['tx_per_second_100s'] = safe_divide(tx_count, time_span_eff, default=0.0)
    state['sol_vol_per_slot_100s'] = safe_divide(state.get('total_sol_vol_100s', 0.0), slot_span_eff, default=0.0)
    state['sol_vol_per_second_100s'] = safe_divide(state.get('total_sol_vol_100s', 0.0), time_span_eff, default=0.0)
    state['lp_sol_rate_per_slot_100s'] = safe_divide(state['lp_sol_change_100s'], slot_span_eff, default=0.0)
    state['lp_sol_rate_per_second_100s'] = safe_divide(state['lp_sol_change_100s'], time_span_eff, default=0.0)
    state['new_wallet_rate_per_slot_100s'] = safe_divide(state['unique_wallets_count_100s'], slot_span_eff, default=0.0)
    state['new_wallet_rate_per_second_100s'] = safe_divide(state['unique_wallets_count_100s'], time_span_eff, default=0.0)

    # --- NEW V2 Features Finalization ---
    tx_first_half = state.get('tx_count_first_half', 0)
    tx_second_half = state.get('tx_count_second_half', 0)
    sol_vol_first_half = state.get('sol_vol_first_half', 0.0)
    sol_vol_second_half = state.get('sol_vol_second_half', 0.0)
    lp_sol_mid = state.get('lp_sol_at_half_way_mark', lp_sol_start) # Default to start if not set

    state['sol_vol_ratio_half_100s'] = safe_divide(sol_vol_second_half, sol_vol_first_half)
    state['tx_count_ratio_half_100s'] = safe_divide(tx_second_half, tx_first_half)

    lp_sol_change_first_half = lp_sol_mid - lp_sol_start
    lp_sol_change_second_half = lp_sol_end - lp_sol_mid
    state['lp_sol_change_first_half'] = lp_sol_change_first_half
    state['lp_sol_change_second_half'] = lp_sol_change_second_half
    state['lp_sol_change_ratio_half_100s'] = safe_divide(lp_sol_change_second_half, lp_sol_change_first_half)

    wallets_first_half_set = state.get('wallets_first_half_set', set())
    # 'wallets_second_half_set' in original accumulation actually stored ALL wallets seen in the window.
    wallets_all_seen_in_window_set = state.get('wallets_second_half_set', set())
    unique_wallets_first_half_count = len(wallets_first_half_set)
    unique_new_wallets_second_half_count = len(wallets_all_seen_in_window_set - wallets_first_half_set)
    state['unique_wallets_first_half_count_100s'] = unique_wallets_first_half_count
    state['unique_new_wallets_second_half_count_100s'] = unique_new_wallets_second_half_count
    state['unique_wallets_ratio_half_100s'] = safe_divide(unique_new_wallets_second_half_count, unique_wallets_first_half_count)

    state['lp_sol_per_buy_sol_100s'] = safe_divide(state['lp_sol_change_100s'], state.get('buy_sol_vol_100s', 0.0))
    state['net_sol_efficiency_100s'] = safe_divide(state['lp_sol_change_100s'], state['net_sol_vol_100s'])
    state['price_change_per_net_sol_100s'] = safe_divide(state.get('price_change_pct_100s', np.nan), state['net_sol_vol_100s'])
    state['sell_pressure_ratio_100s'] = safe_divide(state.get('sell_sol_vol_100s', 0.0), state.get('total_sol_vol_100s', 0.0), default=0.0)

    max_lp_sol_100s = state.get('max_lp_sol_100s', 0.0)
    if max_lp_sol_100s == 0 and lp_sol_end == 0: # If never had LP, sustainability is undefined or 1.
        state['lp_sol_sustainability_100s'] = 1.0 # Or np.nan, depending on desired behavior
    else:
        state['lp_sol_sustainability_100s'] = safe_divide(lp_sol_end, max_lp_sol_100s, default=np.nan)


    state['avg_fee_per_sol_100s'] = safe_divide(state.get('total_fee_100s', 0.0), state.get('total_sol_vol_100s', 0.0))
    state['avg_gas_efficiency_100s'] = safe_divide(state.get('total_gas_consumed_100s', 0.0), state.get('total_provided_gas_limit_100s', 0.0))

    # --- Clean up intermediate structures (sets, specific temp fields) ---
    keys_to_remove_final = [
        'unique_wallets_set', 'unique_buyers_set', 'unique_sellers_set',
        'wallets_first_half_set', 'wallets_second_half_set', # This was wallets_all_seen
        'lp_sol_at_half_way_mark',
        'sum_tx_idx_100s' # sum_tx_idx used for avg, can be removed after avg is calculated
    ]
    for key_to_del in keys_to_remove_final:
        if key_to_del in state:
            del state[key_to_del]

    state['mint'] = mint_key # Ensure mint is a column in the final dict
    return state

# </step3_helpers>

# <step3>
# --- Step 3: Process Chunks, Accumulate, Finalize, and Write Incrementally ---
print(f"Starting chunk processing and feature generation...")
start_time_step3 = time.time()
total_relevant_rows = 0
processed_chunks_count = 0

# Clear previous data if re-running
features_per_mint = {} # This dictionary will store the raw accumulated data

# Ensure mints_info_target is loaded and creation_slots_map is created
# Example: creation_slots_map = mints_info_target.set_index('mint')['target_slot_start'].to_dict()
# Ensure it's available in this scope. If it's large, it should be pre-calculated.
# For this example, let's assume 'mints_info_target' DataFrame is globally available.
if 'mints_info_target' not in globals() or not hasattr(mints_info_target, 'set_index'):
    raise ValueError("DataFrame 'mints_info_target' must be defined and loaded with 'mint' and 'target_slot_start' columns.")
creation_slots_map = mints_info_target.set_index('mint')['target_slot_start'].to_dict()


for i, chunk_filename in enumerate(tqdm(chunk_files, desc="Processing Chunks")):
    try:
        df_chunk = pd.read_csv(
            chunk_filename,
            usecols=COLS_TO_LOAD_FROM_CHUNKS,
            dtype={
                'slot': 'int64', 'tx_idx': 'int16', 'direction': 'category',
                'base_coin': 'object', 'signing_wallet': 'object',
                'base_coin_amount': 'float64', 'quote_coin_amount': 'float64',
                'virtual_token_balance_after': 'float64', 'virtual_sol_balance_after': 'float64',
                'signature': 'object', 'provided_gas_fee': 'float64',
                'provided_gas_limit': 'float64',
                'fee': 'float64', 'consumed_gas': 'float64'
            },
            parse_dates=['block_time'],
            infer_datetime_format=True,
            low_memory=False # Keep as is, or set True if dtypes are fully specified and trusted
        )

        chunk_mints_set = set(df_chunk['base_coin'].unique())
        relevant_mints_in_chunk = chunk_mints_set.intersection(target_mints_dict.keys())
        if not relevant_mints_in_chunk:
             processed_chunks_count += 1
             continue

        df_chunk_filtered = df_chunk[df_chunk['base_coin'].isin(relevant_mints_in_chunk)].copy()
        df_chunk_filtered.sort_values(by=['slot', 'tx_idx'], inplace=True)

        for row in df_chunk_filtered.itertuples(index=False):
            mint = row.base_coin
            slot = row.slot

            if mint in target_mints_dict:
                mint_info = target_mints_dict[mint]
                target_start = mint_info['target_slot_start']
                target_end = mint_info['target_slot_end']
                midpoint_slot = target_start + 50

                if target_start <= slot < target_end:
                    total_relevant_rows += 1
                    direction = row.direction
                    wallet = row.signing_wallet
                    sol_amount = row.quote_coin_amount if pd.notna(row.quote_coin_amount) else 0.0
                    token_amount = row.base_coin_amount if pd.notna(row.base_coin_amount) else 0.0
                    consumed_gas = row.consumed_gas if pd.notna(row.consumed_gas) else 0.0
                    fee = row.fee if pd.notna(row.fee) else 0.0
                    tx_idx = row.tx_idx if pd.notna(row.tx_idx) else 0
                    provided_limit = row.provided_gas_limit if pd.notna(row.provided_gas_limit) else 0.0
                    current_block_time = row.block_time if pd.notna(row.block_time) else pd.NaT
                    current_lp_sol = row.virtual_sol_balance_after if pd.notna(row.virtual_sol_balance_after) else 0.0
                    current_lp_token = row.virtual_token_balance_after if pd.notna(row.virtual_token_balance_after) else 0.0

                    if mint not in features_per_mint:
                        features_per_mint[mint] = {
                            # 'mint': mint, # Will be added during finalization
                            'tx_count_100s': 0,
                            'buy_tx_count_100s': 0, 'sell_tx_count_100s': 0,
                            'total_sol_vol_100s': 0.0, 'buy_sol_vol_100s': 0.0, 'sell_sol_vol_100s': 0.0,
                            'total_token_vol_100s': 0.0, 'buy_token_vol_100s': 0.0, 'sell_token_vol_100s': 0.0,
                            'total_gas_consumed_100s': 0.0, 'total_fee_100s': 0.0,
                            'unique_wallets_set': set(), 'unique_buyers_set': set(), 'unique_sellers_set': set(),
                            'first_slot_100s': slot, 'last_slot_100s': slot,
                            'first_time_100s': current_block_time, 'last_time_100s': current_block_time,
                            'lp_sol_start_100s': current_lp_sol, 'lp_sol_end_100s': current_lp_sol,
                            'lp_token_start_100s': current_lp_token, 'lp_token_end_100s': current_lp_token,
                            'min_lp_sol_100s': current_lp_sol, 'max_lp_sol_100s': current_lp_sol,
                            'min_lp_token_100s': current_lp_token, 'max_lp_token_100s': current_lp_token,
                            'sum_tx_idx_100s': 0, 'max_tx_idx_100s': tx_idx,
                            'max_buy_sol_tx_100s': 0.0, 'max_sell_sol_tx_100s': 0.0,
                            'tx_count_first_half': 0, 'sol_vol_first_half': 0.0,
                            'wallets_first_half_set': set(),
                            'lp_sol_at_half_way_mark': current_lp_sol,
                            'tx_count_second_half': 0, 'sol_vol_second_half': 0.0,
                            'wallets_second_half_set': set(), # This will store ALL wallets during accumulation
                            'total_provided_gas_limit_100s': 0.0,
                        }

                    state = features_per_mint[mint]
                    state['tx_count_100s'] += 1
                    state['unique_wallets_set'].add(wallet)
                    state['wallets_second_half_set'].add(wallet) # Add to this set (which means all wallets in window)
                    state['total_sol_vol_100s'] += sol_amount
                    state['total_token_vol_100s'] += token_amount
                    state['total_gas_consumed_100s'] += consumed_gas
                    state['total_fee_100s'] += fee
                    state['sum_tx_idx_100s'] += tx_idx
                    state['max_tx_idx_100s'] = max(state['max_tx_idx_100s'], tx_idx)
                    state['total_provided_gas_limit_100s'] += provided_limit

                    state['last_slot_100s'] = max(state['last_slot_100s'], slot)
                    if pd.notna(current_block_time):
                        state['last_time_100s'] = max(state['last_time_100s'], current_block_time)
                        if pd.isna(state['first_time_100s']):
                             state['first_time_100s'] = current_block_time

                    state['lp_sol_end_100s'] = current_lp_sol
                    state['lp_token_end_100s'] = current_lp_token
                    state['min_lp_sol_100s'] = min(state['min_lp_sol_100s'], current_lp_sol)
                    state['max_lp_sol_100s'] = max(state['max_lp_sol_100s'], current_lp_sol)
                    state['min_lp_token_100s'] = min(state['min_lp_token_100s'], current_lp_token)
                    state['max_lp_token_100s'] = max(state['max_lp_token_100s'], current_lp_token)

                    if slot < midpoint_slot:
                        state['tx_count_first_half'] += 1
                        state['sol_vol_first_half'] += sol_amount
                        state['wallets_first_half_set'].add(wallet)
                        state['lp_sol_at_half_way_mark'] = current_lp_sol
                    else:
                        state['tx_count_second_half'] += 1
                        state['sol_vol_second_half'] += sol_amount

                    if direction == 'buy':
                        state['buy_tx_count_100s'] += 1
                        state['buy_sol_vol_100s'] += sol_amount
                        state['buy_token_vol_100s'] += token_amount
                        state['unique_buyers_set'].add(wallet)
                        state['max_buy_sol_tx_100s'] = max(state['max_buy_sol_tx_100s'], sol_amount)
                    elif direction == 'sell':
                        state['sell_tx_count_100s'] += 1
                        state['sell_sol_vol_100s'] += sol_amount
                        state['sell_token_vol_100s'] += token_amount
                        state['unique_sellers_set'].add(wallet)
                        state['max_sell_sol_tx_100s'] = max(state['max_sell_sol_tx_100s'], sol_amount)

        processed_chunks_count += 1
        # Optional: delete df_chunk variables and gc.collect() here if chunks are very large
        del df_chunk, df_chunk_filtered, chunk_mints_set, relevant_mints_in_chunk
        if processed_chunks_count % 20 == 0: # Reduce frequency of GC
             gc.collect()


    except FileNotFoundError:
        print(f"Warning: Chunk file not found: {chunk_filename}")
    except pd.errors.EmptyDataError:
         print(f"Warning: Chunk file is empty: {chunk_filename}")
         processed_chunks_count += 1 # Still count as processed for progress
    except Exception as e:
        print(f"ERROR processing chunk {i} ({chunk_filename}): {type(e).__name__} - {e}")
    # No intermediate saving/pickling here

end_time_accumulation = time.time()
print(f"\nChunk processing and data accumulation finished. Time taken: {end_time_accumulation - start_time_step3:.2f} seconds.")
print(f"Total relevant transaction rows processed: {total_relevant_rows}")
print(f"Number of mints with features accumulated: {len(features_per_mint)}")

# --- Finalize features and write to Parquet in batches ---
print("\nFinalizing features and writing to Parquet...")
OUTPUT_PARQUET_PATH = os.path.join(path, 'engineered_features_100s_v2.parquet')
# Adjust BATCH_SIZE_FOR_WRITING based on typical number of mints and available RAM
# A smaller batch size uses less RAM per batch but means more frequent I/O.
BATCH_SIZE_FOR_WRITING = 10000  # Number of mints to process before writing a batch

pq_writer = None
finalized_features_batch_list = []
processed_mints_for_writing = 0
total_mints_to_process = len(features_per_mint)

# Make a list of items to iterate over, allowing features_per_mint to be deleted
# This can free up significant memory before the finalization loop if features_per_mint is large.
mint_items_to_finalize = list(features_per_mint.items())
print(f"Deleting in-memory 'features_per_mint' dictionary ({len(features_per_mint)} items) before finalization.")
del features_per_mint
gc.collect()
print("In-memory 'features_per_mint' deleted.")


# Manually delete the output file if it exists to prevent appending to old data from a failed previous run
if os.path.exists(OUTPUT_PARQUET_PATH):
    print(f"Removing existing output file: {OUTPUT_PARQUET_PATH}")
    os.remove(OUTPUT_PARQUET_PATH)

for mint_key, mint_raw_data_dict in tqdm(mint_items_to_finalize, desc="Finalizing and Writing Features"):
    finalized_dict = finalize_mint_features_dict(mint_key, mint_raw_data_dict, creation_slots_map)
    if finalized_dict:
        finalized_features_batch_list.append(finalized_dict)

    processed_mints_for_writing += 1

    if len(finalized_features_batch_list) >= BATCH_SIZE_FOR_WRITING or processed_mints_for_writing == total_mints_to_process:
        if finalized_features_batch_list:
            batch_df = pd.DataFrame(finalized_features_batch_list)
            batch_df = batch_df.replace([np.inf, -np.inf], np.nan) # Redundant if safe_divide is perfect, but safe

            try:
                arrow_table = pa.Table.from_pandas(batch_df, preserve_index=False)
                if pq_writer is None:
                    pq_writer = pq.ParquetWriter(OUTPUT_PARQUET_PATH, arrow_table.schema)
                pq_writer.write_table(arrow_table)
            except pa.ArrowInvalid as e:
                print(f"ArrowInvalid error during batch write: {e}")
                print("Problematic batch_df head:")
                print(batch_df.head())
                print("Problematic batch_df dtypes:")
                print(batch_df.dtypes)
                # Potentially save problematic batch for inspection
                # batch_df.to_csv(os.path.join(path, f"error_batch_{processed_mints_for_writing}.csv"), index=False)
                raise e # Re-raise after logging

            finalized_features_batch_list = [] # Clear the batch list for the next set
            # print(f"  Written batch to {OUTPUT_PARQUET_PATH}. {processed_mints_for_writing}/{total_mints_to_process} mints processed.")
            gc.collect() # Collect garbage after processing a batch and creating a DataFrame

if pq_writer:
    pq_writer.close()
    print(f"All features successfully written to {OUTPUT_PARQUET_PATH}")
else:
    if total_mints_to_process > 0 :
        print("Warning: Parquet writer was not initialized. No data written, but mints were processed.")
    else:
        print("No mints to process. No features were generated or written.")


del mint_items_to_finalize # Explicitly delete the list
if 'finalized_features_batch_list' in locals(): del finalized_features_batch_list # Should be empty
gc.collect()

end_time_step3_final = time.time()
print(f"Total Step 3 (accumulation, finalization, writing) time: {end_time_step3_final - start_time_step3:.2f} seconds.")
# </step3>

Starting chunk processing and feature generation...


Processing Chunks:   0%|          | 0/41 [00:00<?, ?it/s]

<ipython-input-27-2811ca8958a2>:184: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_chunk = pd.read_csv(
<ipython-input-27-2811ca8958a2>:184: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_chunk = pd.read_csv(
<ipython-input-27-2811ca8958a2>:184: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_chunk = pd.read_csv(
<ipython-input-2


Chunk processing and data accumulation finished. Time taken: 808.90 seconds.
Total relevant transaction rows processed: 16554095
Number of mints with features accumulated: 1061444

Finalizing features and writing to Parquet...
Deleting in-memory 'features_per_mint' dictionary (1061444 items) before finalization.
In-memory 'features_per_mint' deleted.


Finalizing and Writing Features:   0%|          | 0/1061444 [00:00<?, ?it/s]

All features successfully written to /content/drive/MyDrive/Kaggle/engineered_features_100s_v2.parquet
Total Step 3 (accumulation, finalization, writing) time: 2118.77 seconds.


In [ ]:
# <step4>
# --- Step 4: Load and Inspect Final Features (Now Optional) ---
# The features are already generated and saved by the new Step 3.
# This step is now primarily for loading and inspecting if needed.

print("\n--- Step 4: Inspecting Final Features ---")
FINAL_FEATURES_PATH = os.path.join(path, 'engineered_features_100s_v2.parquet') # Same path as used in Step 3

if not os.path.exists(FINAL_FEATURES_PATH):
    print(f"ERROR: Final features file not found at {FINAL_FEATURES_PATH}.")
    print("This means either Step 3 failed, or no data was written.")
else:
    try:
        print(f"Loading final features from {FINAL_FEATURES_PATH} for inspection...")
        features_df = pd.read_parquet(FINAL_FEATURES_PATH)
        print(f"Successfully loaded features DataFrame. Shape: {features_df.shape}")

        if not features_df.empty:
            print("\nGenerated Features Head (V2):")
            print(features_df.head())

            # print("\nGenerated Features Info (V2):")
            # features_df.info(verbose=True, show_counts=True) # Can be very verbose for many columns

            print("\nChecking for NaNs per column (Top 20):")
            nan_counts = features_df.isnull().sum()
            print(nan_counts[nan_counts > 0].sort_values(ascending=False).head(20))
        else:
            print("Loaded features DataFrame is empty.")

    except Exception as e:
        print(f"ERROR loading or inspecting final features: {e}")
        # raise e # Or handle differently

# The intermediate pickle file and its loading logic are no longer present here.
# All feature calculation and saving happened in the modified Step 3.
# </step4>


--- Step 4: Inspecting Final Features ---
Loading final features from /content/drive/MyDrive/Kaggle/engineered_features_100s_v2.parquet for inspection...
Successfully loaded features DataFrame. Shape: (1061444, 89)

Generated Features Head (V2):
   tx_count_100s  buy_tx_count_100s  sell_tx_count_100s  total_sol_vol_100s  \
0              3                  2                   1        5.656629e+09   
1              1                  1                   0        4.000000e+09   
2              3                  2                   1        2.574964e+09   
3              1                  1                   0        2.000000e+09   
4             13                  8                   5        7.874963e+09   

   buy_sol_vol_100s  sell_sol_vol_100s  total_token_vol_100s  \
0      3.694123e+09       1.962506e+09          1.767267e+14   
1      4.000000e+09       0.000000e+00          1.262353e+14   
2      1.787482e+09       7.874822e+08          8.606154e+13   
3      2.000000e+09   

In [ ]:
features_df.head()

,mint,tx_count_100s,buy_tx_count_100s,sell_tx_count_100s,total_sol_vol_100s,buy_sol_vol_100s,sell_sol_vol_100s,total_token_vol_100s,buy_token_vol_100s,sell_token_vol_100s,...,last_slot_offset_100s,time_span_seconds_100s,tx_per_slot_100s,tx_per_second_100s,sol_vol_per_slot_100s,sol_vol_per_second_100s,lp_sol_rate_per_slot_100s,lp_sol_rate_per_second_100s,new_wallet_rate_per_slot_100s,new_wallet_rate_per_second_100s
0,BmTDA5HqcemLkEgpyK25sDhbvk652CTXjdWEa8fLpump,3,2,1,5.656629e+09,3.694123e+09,1.962506e+09,1.767267e+14,1.176405e+14,5.908613e+13,...,53,21.0,0.055556,0.142857,1.047524e+08,2.693633e+08,3.114106e+07,8.007702e+07,0.037037,0.095238
1,4FJwryCAMMePNeWw9LTBXXfXABdkKcAxkTT6h9pdpump,1,1,0,2.000000e+09,2.000000e+09,0.000000e+00,6.706250e+13,6.706250e+13,0.000000e+00,...,0,0.0,1.000000,1.000000,2.000000e+09,2.000000e+09,0.000000e+00,0.000000e+00,1.000000,1.000000
2,4rbGH5peYV3FvCfncfsfwtBNwpjiG36nZtvxoPZ9pump,13,8,5,7.874963e+09,4.390652e+09,3.484311e+09,2.427691e+14,1.371176e+14,1.056515e+14,...,94,37.0,0.136842,0.351351,8.289435e+07,2.128368e+08,4.277271e+06,1.098218e+07,0.084211,0.216216
3,6Mx4fnEPWpRLKtxY8VzVXo3CWwu2dvGp9BSJMWfVpump,1,1,0,4.000000e+09,4.000000e+09,0.000000e+00,1.262353e+14,1.262353e+14,0.000000e+00,...,0,0.0,1.000000,1.000000,4.000000e+09,4.000000e+09,0.000000e+00,0.000000e+00,1.000000,1.000000
4,DeXchZLMzFm9nfvkfTKiXvBWWEpGMF6VDzvbMB6mpump,3,2,1,2.574964e+09,1.787482e+09,7.874822e+08,8.606154e+13,6.033722e+13,2.572432e+13,...,12,5.0,0.230769,0.600000,1.980742e+08,5.149929e+08,7.692308e-02,2.000000e-01,0.153846,0.400000
